In [1]:
import pandas as pd

In [2]:
filepath = '../data/processed/howstat/fall_of_wickets/'

In [3]:
file = 'fow_2400.csv'

In [4]:
df = pd.read_csv(filepath+file, index_col=0, parse_dates=[2], infer_datetime_format=True)

In [5]:
df.head(10)

,MatchId,MatchDate,MatchInnings,Team,TeamInnings,Wicket,Runs,Player
0,2400,2020-01-03,1,England,1st,1,8,Crawley
1,2400,2020-01-03,1,England,1st,2,63,Sibley
2,2400,2020-01-03,1,England,1st,3,105,Root
3,2400,2020-01-03,1,England,1st,4,127,Denly
4,2400,2020-01-03,1,England,1st,5,185,Stokes
5,2400,2020-01-03,1,England,1st,6,221,Buttler
6,2400,2020-01-03,1,England,1st,7,231,Curran
7,2400,2020-01-03,1,England,1st,8,231,Bess
8,2400,2020-01-03,1,England,1st,9,234,Broad
9,2400,2020-01-03,1,England,1st,10,269,Anderson


In [6]:
df.groupby(['MatchId','MatchInnings'])['Wicket', 'Runs'].max()

<ipython-input-6-27153cc069be>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df.groupby(['MatchId','MatchInnings'])['Wicket', 'Runs'].max()


Wicket  Runs
MatchId MatchInnings              
2400    1                 10   269
        2                 10   223
        3                  8   375
        4                 10   248

Let's say a batting collapse is defined as losing at least 3 wickets for at most 30 runs.
We want to know for each MatchId-MatchInnings whether that innings is a batting collapse.

Say we have FoW:
1-20
2-40
3-41

This is a batting collapse.
One approach:
If we know we have only 5 wickets lost:
Check each group of 3 wickets. Groups will be: 5-2, 4-1, 3-0.
Convert the Runs into an array, with index=0 wickets.


Test cases for groups of 3: 
Wickets 1,2,3 is not a collapse: 105 runs
Wickets 4,5,6 is not a collapse: 94 runs
Wickets 6,7,8 is a collapse: 10 runs
Wickets 7,8,9 is a collapse: 3 runs
Wickets 8,9,10 is not a collapse: 38 runs

Test cases for groups of 2:


In [97]:
l_runs = list(df[df.MatchInnings==2].Runs)
print(l_runs)

[26, 38, 40, 157, 191, 200, 207, 215, 215, 223]


In [98]:
l_runs.insert(0,0)

In [99]:
l_runs

[0, 26, 38, 40, 157, 191, 200, 207, 215, 215, 223]

In [30]:
l_test = [1,2,3]
run_difference = l_runs[l_test[-1]] - l_runs[l_test[0]]
print(run_difference)


97


In [45]:
from itertools import tee, islice

def nwise(iterable, n):                                                      
    iters = tee(iterable, n)                                                     
    for i, it in enumerate(iters):                                               
        next(islice(it, i, i), None)                                               
    return zip(*iters)   

list(nwise(l_runs, n=2))

[(0, 8),
 (8, 63),
 (63, 105),
 (105, 127),
 (127, 185),
 (185, 221),
 (221, 231),
 (231, 231),
 (231, 234),
 (234, 269)]

In [50]:
l_runs

[0, 8, 63, 105, 127, 185, 221, 231, 231, 234, 269]

In [100]:
l_runs

[0, 26, 38, 40, 157, 191, 200, 207, 215, 215, 223]

In [101]:
n = 3
for i in range(n,len(l_runs)):
    print(f'Start: {i-n}')
    print(f'End: {i}')
    
    if i-n == 0:
        run_difference = l_runs[i]
    else:
        run_difference = l_runs[i] - l_runs[i-n-1]
    print(f'Run difference: {run_difference}')
    l_positions_involved = [x for x in range(i-n+1, i+1)]
    print(f'Positions involved: {l_positions_involved}')
    print('\n')

Start: 0
End: 3
Run difference: 40
Positions involved: [1, 2, 3]


Start: 1
End: 4
Run difference: 157
Positions involved: [2, 3, 4]


Start: 2
End: 5
Run difference: 165
Positions involved: [3, 4, 5]


Start: 3
End: 6
Run difference: 162
Positions involved: [4, 5, 6]


Start: 4
End: 7
Run difference: 167
Positions involved: [5, 6, 7]


Start: 5
End: 8
Run difference: 58
Positions involved: [6, 7, 8]


Start: 6
End: 9
Run difference: 24
Positions involved: [7, 8, 9]


Start: 7
End: 10
Run difference: 23
Positions involved: [8, 9, 10]




In [93]:
def containsCollapse(df):
    
    l_runs = list(df.Runs)
    l_runs.insert(0,0)
    
    n = 3
    n_collapses = 0
    d_collapses = {}
    for i in range(n,len(l_runs)):

        if i-n == 0:
            run_difference = l_runs[i]
        else:
            run_difference = l_runs[i] - l_runs[i-n-1]
        
        if run_difference <= 30:
            n_collapses += 1
            l_positions_involved = [x for x in range(i-n+1, i+1)]
            d_collapses[n_collapses] = l_positions_involved
    
    return n_collapses, d_collapses
            
    # for each collapse, want to store:
    # - number of runs lost
    # - positions involved
    # - (later on) batsman involved
    # - (later on) bowlers involved

In [95]:
df.groupby(['MatchId','MatchInnings']).apply(containsCollapse)

MatchId  MatchInnings
2400     1                                          (0, {})
         2               (2, {1: [7, 8, 9], 2: [8, 9, 10]})
         3                                          (0, {})
         4                             (1, {1: [8, 9, 10]})
dtype: object

In [65]:
df

,MatchId,MatchDate,MatchInnings,Team,TeamInnings,Wicket,Runs,Player
0,2400,2020-01-03,1,England,1st,1,8,Crawley
1,2400,2020-01-03,1,England,1st,2,63,Sibley
2,2400,2020-01-03,1,England,1st,3,105,Root
3,2400,2020-01-03,1,England,1st,4,127,Denly
4,2400,2020-01-03,1,England,1st,5,185,Stokes
5,2400,2020-01-03,1,England,1st,6,221,Buttler
6,2400,2020-01-03,1,England,1st,7,231,Curran
7,2400,2020-01-03,1,England,1st,8,231,Bess
8,2400,2020-01-03,1,England,1st,9,234,Broad
9,2400,2020-01-03,1,England,1st,10,269,Anderson
